# Nearest neighbor search (NNS)

## k Nearest Neighbor (kNN)
kNNは3次元空間上の点群に対して使用することがあります。3次元座標値を持つ点群に対してkNNを使用した場合、点群中のある点aに近いk個の点を特定することができます。これにより、aの周りにある点の値を参照して特徴量の生成や畳み込み処理などを行うことができる様になります。

ここで実装されているkNNは以下のプロセスを実行します。

1. N個の点群AとM個の点群Bから、互いの点ごとの相対距離を取得します。相対距離は、(N, M)の配列で表されます。
    1. 当然、AとBの点群が同じである場合もあります。AがN個の点を持ちなおかつA=Bの場合は、配列は(N, N)の形となり、距離行列と同じものとなります。
2. 1の行列に基づき、Aの各点において、自身の点から近いBの点をk個取得します。結果として(N, k)のインデックス配列を得ることができます。インデックスは、Bの点群のインデックスです。
    1. 以下の実装では、インデックス配列と相対距離の配列を返します。

ひとまず、実際にkNNがどの様に機能するか確認してみましょう。以下は本subsectionで使うパッケージです。

In [1]:
# for kNN
import numpy as np
from tutlibs.nns import k_nearest_neighbors
from tutlibs.utils import grouping

# for description
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-19 14:22:45,964 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2021-09-19 14:22:45,964 - utils - NumExpr defaulting to 8 threads.


はじめに、点群Bから一部サンプリングした点を点群Aとして扱います。

In [2]:
# a point cloud
B, _, _ = io.read('../data/bunny_pc.ply')
B_colors = np.tile(np.array([[0, 0, 1]]), (len(B), 1))

# sampling
A = B[::500]
A_colors = np.tile(np.array([[1, 0, 0]]), (len(A), 1))

# visualization
visualizer.k3d(np.vstack([B, A]), np.vstack([B_colors, A_colors]), point_size=0.02)

Output()

次に、Aの各点から見たBのk近傍を表示します。

In [ ]:
k = 10
idx, dist = k_nearest_neighbors(A, B, k)
knn_points = grouping(B, idx)
knn_points = knn_points.reshape(len(A)*k, 3)
knn_colors = np.tile(np.array([[0, 1, 0]]), (len(knn_points), 1))
visualizer.k3d(np.vstack([B, knn_points, A]), np.vstack([B_colors, knn_colors, A_colors]), point_size=0.02)


Output()

## Radius Nearest Neighbor

使用するパッケージは以下のとおりです。

In [1]:
import numpy as np
from tutlibs.nns import radius_nearest_neighbors
from tutlibs.utils import grouping
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-18 13:54:05,669 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2021-09-18 13:54:05,669 - utils - NumExpr defaulting to 8 threads.


コードは以下のとおりです。

In [2]:
# a point cloud
B, _, _ = io.read('../data/bunny_pc.ply')
B_colors = np.tile(np.array([[0, 0, 1]]), (len(B), 1))

# sampling
A = B[::500]
A_colors = np.tile(np.array([[1, 0, 0]]), (len(A), 1))

# visualization
visualizer.k3d(np.vstack([B, A]), np.vstack([B_colors, A_colors]), point_size=0.02)

Output()

In [ ]:
k = 10
r = 0.005

# Get knn indexs and radius nearest neighbor mask
idxs, min_dists, radius_masks = radius_nearest_neighbors(A, B, r, k)
knn_points = grouping(B, idxs)

# Get radius nearest neighbor (rnn) points using radius mask
num_all_rnn = np.sum(radius_masks)
rnn_points = knn_points[radius_masks] # shape: (N, k, C) -> (num_all_rnn, C)
rnn_colors = np.tile(np.array([[0, 1, 0]]), (len(rnn_points), 1))

visualizer.k3d(np.vstack([B, rnn_points, A]), np.vstack([B_colors, rnn_colors, A_colors]), point_size=0.02)


Output()